# Libraries 

In [1]:
import pandas as pd 
from wordcloud import WordCloud
from collections import defaultdict

# File load

In [2]:
# Load file
orders = pd.read_csv("input_test_info.csv")

In [3]:
orders.head()

,EAN,TITRE,AUTEUR,EDITEUR,PRIX,DISPO,QTE,TOTAL
0,9782723441995,RAGHNAROK - TOME 03 - TERREURS DE LA NATURE,BOULET,GLENAT,"10,95",1,1,"10,95"
1,9782723442008,LA RUBRIQUE SCIENTIFIQUE - TOME 02,BOULET,GLENAT,"10,95",7,1,"10,95"
2,9782723442022,"WOMOKS - TOME 03 - ALBON, LES BRUTES ET LES TR...",BOULET/RENO,GLENAT,"10,95",1,1,"10,95"
3,9782723442039,TENZING ET LES SHERPAS DE L'EVEREST,TENZING,GLENAT,"19,99",7,1,"19,99"
4,9782723442077,BLACK CAT - TOME 02,YABUKI KENTARO,GLENAT,"6,9",7,1,"6,9"


# Transformations
- Cast the "total" column to float 
- Cast the "price" column to float

In [4]:
# Cast the price values to float
orders["TOTAL"] = orders.TOTAL.apply(lambda x: float(str(x).replace(",",".")))
orders["PRIX"] = orders.PRIX.apply(lambda x: float(str(x).replace(",",".")))
orders.head()

,EAN,TITRE,AUTEUR,EDITEUR,PRIX,DISPO,QTE,TOTAL
0,9782723441995,RAGHNAROK - TOME 03 - TERREURS DE LA NATURE,BOULET,GLENAT,10.95,1,1,10.95
1,9782723442008,LA RUBRIQUE SCIENTIFIQUE - TOME 02,BOULET,GLENAT,10.95,7,1,10.95
2,9782723442022,"WOMOKS - TOME 03 - ALBON, LES BRUTES ET LES TR...",BOULET/RENO,GLENAT,10.95,1,1,10.95
3,9782723442039,TENZING ET LES SHERPAS DE L'EVEREST,TENZING,GLENAT,19.99,7,1,19.99
4,9782723442077,BLACK CAT - TOME 02,YABUKI KENTARO,GLENAT,6.90,7,1,6.90


# Calculations

In [5]:
# Initialize the counter
booked = 0 
# Using a defaultdict because
# the values are automatically 0
# when not modified
not_booked = defaultdict(int)
total = 0

# Loop through the rows
for i in range(len(orders)):
    # Get the parameters
    bar_code = orders.loc[i,"EAN"]
    dispo = orders.loc[i, "DISPO"]
    qte = orders.loc[i,"QTE"]
    price = orders.loc[i,"PRIX"]
    
    # Check for the availability
    if qte > dispo:
        booked+= dispo
        # Update the total price
        total+= (price * dispo)
        # Update the availability 
        not_booked[bar_code]+= qte - dispo
        orders[orders["EAN"]==bar_code]["DISPO"] = 0
    else: 
        booked+= qte
        # Update the total price
        total+= (price * qte)
        # Update the availability 
        orders[orders["EAN"]==bar_code]["DISPO"] = dispo - qte

C:\Users\User\AppData\Local\Temp\ipykernel_20732\99894656.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders[orders["EAN"]==bar_code]["DISPO"] = dispo - qte
C:\Users\User\AppData\Local\Temp\ipykernel_20732\99894656.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders[orders["EAN"]==bar_code]["DISPO"] = 0


# Get the stats 

## Author stats

In [6]:
authors = orders["AUTEUR"]
# Get the count for each author
value_counts = dict(authors.value_counts())
# Get the author count
author_count = len(authors)
# Initialize the percentage list
author_perc = dict()
# Get the percentages 
for key,value in value_counts.items():
    author_perc[key] = round(value*100/author_count,2)

## Editor stats 

In [7]:
editors = orders["EDITEUR"]
# Get the count for each editor
value_counts = dict(editors.value_counts())
# Get the author count
editor_count = len(editors)
# Initialize the percentage list
editor_perc = dict()
# Get the percentages 
for key,value in value_counts.items():
    editor_perc[key] = round(value*100/editor_count,2)

## Word stats

In [8]:
# Excluding the word "TOME"
wordcloud = WordCloud(stopwords=["TOME"],collocations=False)
# Get the corpus
text = " ".join(orders["TITRE"])
# Generate the wordcloud statistics
wordcloud.generate_from_text(text)

In [9]:
# Retrieve the first 5 words
first_five = wordcloud.words_
first_five = list(first_five.keys())[0:5]
first_five

['LE', 'DE', 'LA', 'DU', 'ET']

In [10]:
column_names = ["QTY TOTAL","QTY NP", "PRIX TOTAL", "STAT-AUT", "STAT-EDT", "EXPR REG"]


# Generate the output

In [11]:
# Intialize the resulting dataframe 
# as a dict
result = dict()
result["QTY TOTAL"] = booked
result["QTY NP"] = []
result["PRIX TOTAL"] = total
result["STAT-AUT"] = []
result["STAT-EDT"] = []
result["EXPR REG"] = first_five

# QTY NP
for i in range(len(orders)):
    # Get the values
    bar_code = orders.loc[i,"EAN"]
    nb = not_booked[bar_code]
    
    # Create the string value
    string = f"{bar_code}-{nb}"
    # Append to the list
    result["QTY NP"].append(string)

# Author perc
for key,value in author_perc.items():
    # Create the string value 
    string = f"{key}-{value}"
    result["STAT-AUT"].append(string)

# Editor perc  
for key,value in editor_perc.items():
    # Create the string value 
    string = f"{key}-{value}"
    result["STAT-EDT"].append(string)

# Create the output DataFrame

In [12]:
output = pd.DataFrame(columns=column_names)
output.head()

,QTY TOTAL,QTY NP,PRIX TOTAL,STAT-AUT,STAT-EDT,EXPR REG


In [13]:
# Assign the column values
# If there's a list, display the values 
# with a line separator
output["QTY TOTAL"] = [result["QTY TOTAL"]]
output["QTY NP"] = "\n".join(result["QTY NP"])
output["PRIX TOTAL"] = [result["PRIX TOTAL"]]
output["STAT-AUT"] = "\n".join(result["STAT-AUT"])
output["STAT-EDT"] = "\n".join(result["STAT-EDT"])
output["EXPR REG"] = "\n".join(result["EXPR REG"])

In [14]:
output.head()

,QTY TOTAL,QTY NP,PRIX TOTAL,STAT-AUT,STAT-EDT,EXPR REG
0,915,9782723441995-0\n9782723442008-0\n978272344202...,11156.45,ADACHI MITSURU-4.44\nKAMIO YOKO-3.44\nMASHIMA ...,GLENAT-100.0,LE\nDE\nLA\nDU\nET


# Save the output

In [15]:
output.to_csv("jap_one.csv",index=False,sep=",",line_terminator="\n")